In [2]:
from os import listdir, system, path

"""Python WPS execute"""
from owslib.wps import WebProcessingService, monitorExecution, printInputOutput
#wps_url = "http://birdhouse-lsce.extra.cea.fr:8093/wps"
wps_url = "http://localhost:8093/wps"
wps = WebProcessingService(url=wps_url, verbose=False)

In [3]:
print wps.identification.title

for process in wps.processes:
    print '%s : \t %s' % (process.identifier, process.abstract)

Flyingpigeon
visualisation : 	 Just testing a nice script to visualise some variables
sdm : 	 Species distribution model (SDM)
weatherregimes : 	 Weather Regimes based on pressure patterns (kmean method)
extractpoints : 	 Extract Timeseries for specified coordinates from grid data
segetalflora : 	 Species biodiversity of segetal flora. Imput files: variable:tas , domain: EUR-11 or EUR-44
indices_single : 	 This process calculates climate indices based on one single variable.
indices_percentile : 	 This process calculates climate indices based on one single variable and based on percentils of a referece period.
subset_countries : 	 This process returns only the given polygon from input netCDF files.
eobs_to_cordex : 	 downloads EOBS data in adaped CORDEX format
ensembleRobustness : 	 Calculates the robustness as the ratio of noise to signal in an ensemle of timeseries
analogs : 	 Search for days with analog pressure pattern
fetch : 	 This process downloads resources (limited to 50GB)   

In [4]:
p = wps.describeprocess(identifier='weatherregimes')
for input in p.dataInputs:
    printInputOutput(input)
    print '\n'

 identifier=resources, title=NetCDF File, abstract=NetCDF File, data type=ComplexData
 Supported Value: mimeType=application/x-netcdf, encoding=None, schema=None
 Default Value: mimeType=application/x-netcdf, encoding=None, schema=None 
 minOccurs=1, maxOccurs=500


 identifier=BBox, title=Region, abstract=coordinates to define the region: (minlon,minlat,maxlon,maxlat), data type=//www.w3.org/TR/xmlschema-2/#string
 Any value allowed
 Default Value: -80,50,22.5,70 
 minOccurs=0, maxOccurs=1


 identifier=time_region, title=Time region, abstract=Select the months to define the time region (None == whole year will be analysed), data type=//www.w3.org/TR/xmlschema-2/#string
 Allowed Value: 10,11,12,1,2,3
 Allowed Value: 4,5,6,7,8,9
 Allowed Value: 12,1,2
 Allowed Value: 3,4,5
 Allowed Value: 6,7,8
 Allowed Value: 9,10,11
 Allowed Value: None
 Default Value: 12,1,2 
 minOccurs=0, maxOccurs=1


 identifier=observation, title=Observation Data, abstract=Choose an observation dataset for compa

In [5]:
p = '/home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/'
#p = '/home/nils/data/eucleia/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/'
resources = ['file://%s' % path.join(p,f) for f in listdir(p) if 'psl_' in f ]

In [6]:
import requests
r = ''
for i in range(len(resources)):
    r= '%sresources={resource%s};' % (r,i)
print r 

resources={resource0};resources={resource1};resources={resource2};resources={resource3};resources={resource4};resources={resource5};


In [7]:
async_req_url = "{wps_url}?" +\
    "request=Execute" +\
    "&service=WPS" +\
    "&version=1.0.0" +\
    "&identifier=weatherregimes" +\
    "&DataInputs="+r+\
    "&storeExecuteResponse=true" +\
    "&status=true"
  
  
url=async_req_url.format(
    wps_url=wps_url,
    resource0=resources[0],
    resource1=resources[1],
    resource2=resources[2],    
    resource3=resources[3],
    resource4=resources[4],
    resource5=resources[5],
)
print url   

http://localhost:8093/wps?request=Execute&service=WPS&version=1.0.0&identifier=weatherregimes&DataInputs=resources=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/psl_day_HadGEM3-A-N216_historical_r1i1p1_19800101-19891230.nc;resources=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/psl_day_HadGEM3-A-N216_historical_r1i1p1_19600101-19691230.nc;resources=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/psl_day_HadGEM3-A-N216_historical_r1i1p1_19700101-19791230.nc;resources=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/psl_day_HadGEM3-A-N216_historical_r1i1p1_20100101-20131230.nc;resources=file:///home/estimr1/EUCLEIA/gws-access.ceda.ac.uk/public/mohc_shared/EUCLEIA/MOHC/HadGEM3-A-N216/historical/r1i1p1/psl_day_HadGEM3-A

In [8]:
r = requests.get(url)
from lxml import etree
from io import BytesIO
tree = etree.parse(BytesIO(r.content))
print etree.tostring(tree)
status_url = tree.getroot().get("statusLocation")
print status_url

<wps:ExecuteResponse xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-CA" serviceInstance="http://localhost:8093/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0" statusLocation="http://localhost:8090/wpsoutputs/flyingpigeon/pywps-891d907a-0264-11e6-a605-b1dada5c91c5.xml">
    <wps:Process wps:processVersion="0.1">
        <ows:Identifier>weatherregimes</ows:Identifier>
        <ows:Title>Weather Regimes</ows:Title>
        <ows:Abstract>Weather Regimes based on pressure patterns (kmean method)</ows:Abstract>
        <ows:Metadata xlink:title="Weather Regimes" xlink:href="http://localhost:8093/wps?service=WPS&amp;request=DescribeProcess&amp;version=1.0.0&amp;identifier=weatherregimes"

In [9]:
r = requests.get(status_url)
print r.status_code
print r.text

200
<?xml version="1.0" encoding="utf-8"?>
<wps:ExecuteResponse xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsExecute_response.xsd" service="WPS" version="1.0.0" xml:lang="en-CA" serviceInstance="http://localhost:8093/wps?service=WPS&amp;request=GetCapabilities&amp;version=1.0.0" statusLocation="http://localhost:8090/wpsoutputs/flyingpigeon/pywps-891d907a-0264-11e6-a605-b1dada5c91c5.xml">
    <wps:Process wps:processVersion="0.1">
        <ows:Identifier>weatherregimes</ows:Identifier>
        <ows:Title>Weather Regimes</ows:Title>
        <ows:Abstract>Weather Regimes based on pressure patterns (kmean method)</ows:Abstract>
        <ows:Metadata xlink:title="Weather Regimes" xlink:href="http://localhost:8093/wps?service=WPS&amp;request=DescribeProcess&amp;v